## ???

In [ ]:
def transform(resample_df, station_data, weather_data, trip_data, idx):
    df_agg = resample_df
    # извлекаем дни
    df_agg['Date'] = getDate(df_agg, idx, 'd')
    # извлекаем часы
    df_agg['Hour'] = getDate(df_agg, idx, 'h')
    df_agg.eval('net_rate=num_trips_end-num_trips_start', inplace=True)
    df_agg.drop(['num_trips_end', 'num_trips_start'], inplace=True, axis=1)

    # соединяем данные
    # данные по станциям
    df_agg = df_agg.join(station_data.set_index('Station'))
    # погода
    df_agg.reset_index(inplace=True)
    df_agg.set_index(['Date', 'Zip'], inplace=True)

    df_agg = df_agg.join(weather_data) \
                    .reset_index() \
                    .set_index(['Station', 'Datetime'])

    # определяем тип покупателя (!)
    #df_agg = get_dummies(df_agg, 'Subscriber_Type', dropOrigin=True)
    # Определяем время дня
    df_agg['Is_night'] = df_agg['Hour'].apply(lambda h: 1 if h < 5 or h > 20 else 0)
    # Определяем день недели
    df_agg['Day_of_week'] = df_agg['Date'].apply(lambda dt: dt.weekday())
    # Рабочий или выходной
    df_agg['Is_weekday'] = df_agg['Day_of_week'].apply(lambda s: 0 if s in [5, 6] else 1)

    # Сизоны: зима (0), осень (1), лето (2), осень (3)
    df_agg['Season'] = df_agg['Date'].apply(lambda dt: (dt.month % 12 + 3) // 3 - 1)

    df = get_dummies(trip_data, 'Subscriber_Type', dropOrigin=True)
    end = get_stop(df, idx, 'H', 'End_Station', 'End_Date', drop=True, dropId="Trip_ID")
    start = get_stop(df, idx, 'H', 'Start_Station', 'Start_Date', drop=True, dropId="Trip_ID")

    # общий набор
    df_agg = df_agg.join(end).join(start, lsuffix='_end', rsuffix='_start')
    #self.df_agg = self.df_agg.sample(frac=0.1)

    # Net rate за преведующий час
    df_agg['net_rate_previous_hour'] = df_agg.groupby(['Station', 'Date'])['net_rate'] \
                                                .shift(1).fillna(0)
    df_agg = cyclic_feature(df_agg)

    # OHE Events
    df_agg = get_dummies(df_agg,'Events')

    # OHE Station IDs
    df_agg.reset_index(inplace=True)
    df_agg = get_dummies(df_agg, 'Station', prefix='Station')
    df_agg = get_dummies(df_agg, 'Zip', prefix='Zip')

    df_agg.drop(['Hour', 'Day_of_week', 'Season', 'Datetime', 'Date'],
                       inplace=True, axis=1)

    return df_agg


In [1]:
def transform(resample_df, station_data, weather_data, trip_data, idx):
    # этот шаг зачем? (не хотелось переписывать переменную)
    # в памяти 2 объекта ссылаются на одинаковые ячейки в памяти
    #df_agg = resample_df
    
    # 2 похожих действия
    df_agg['Date'] = getDate(df_agg, idx, 'd')
    df_agg['Hour'] = getDate(df_agg, idx, 'h')
    
    # не всем знакомый методо
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.eval.html
    df_agg.eval('net_rate=num_trips_end-num_trips_start', inplace=True)
    
    # drop в середине, встретится ли в конце?
    df_agg.drop(['num_trips_end', 'num_trips_start'], inplace=True, axis=1)
    
    # можно ли представить в виде pandas pipeline:
    # df.method().method().method()  ! если все возвращают объекты DF
    df_agg = (
        df_agg.join(station_data.set_index('Station'))
               .reset_index()
               .set_index(['Date', 'Zip'])
               .join(weather_data)
                .reset_index()
                .set_index(['Station', 'Datetime'])
        )
    
    # почти одинаковое действие, но с разными значениям (4 разные функции)
    df_agg['Is_night'] = df_agg['Hour'].apply(lambda h: 1 if h < 5 or h > 20 else 0)
    df_agg['Day_of_week'] = df_agg['Date'].apply(lambda dt: dt.weekday())
    df_agg['Is_weekday'] = df_agg['Day_of_week'].apply(lambda s: 0 if s in [5, 6] else 1)
    df_agg['Season'] = df_agg['Date'].apply(lambda dt: (dt.month % 12 + 3) // 3 - 1)
    
    # много раз повторяющаяся функция 
    df = get_dummies(trip_data, 'Subscriber_Type', dropOrigin=True)
    end = get_stop(df, idx, 'H', 'End_Station', 'End_Date', drop=True, dropId="Trip_ID")
    start = get_stop(df, idx, 'H', 'Start_Station', 'Start_Date', drop=True, dropId="Trip_ID")
    
    #
    df_agg = df_agg.join(end).join(start, lsuffix='_end', rsuffix='_start')
    df_agg['net_rate_previous_hour'] = df_agg.groupby(['Station', 'Date'])['net_rate'] \
                                                .shift(1).fillna(0)
    
    df_agg = cyclic_feature(df_agg)
    df_agg = get_dummies(df_agg,'Events')
    
    # почему сейчас?
    df_agg.reset_index(inplace=True)
    
    # все dummies сразу?
    df_agg = get_dummies(df_agg, 'Station', prefix='Station')
    df_agg = get_dummies(df_agg, 'Zip', prefix='Zip')

    # все дроп в конце или отдельной функцией 
    df_agg.drop(['Hour', 'Day_of_week', 'Season', 'Datetime', 'Date'],
                       inplace=True, axis=1)

    return df_agg


In [ ]:
# внутри функции, есть функции, с них и начинаем тестирование
# transform - сложная функции из последовательностей -> упростить
# -> построить граф выполнения и посмотреть (найти) на возможности упрощения
getDate()

lambda x: x

get_dummies()

get_stop()

cyclic_feature()


![](graph.jpg)

In [2]:
# для обучения внутри команды + для менеджеров
# https://checkio.org/

## Ссылка на один объект

In [5]:
def best_func(some_names, outputs = []):
    splt = some_names.split(',')
    [outputs.append(i.strip()) for i in splt]
    return outputs

In [6]:
best_func('Roma, Maria', [])

['Roma', 'Maria']

In [7]:
best_func('Roma, Maxim, Dmitriy')

['Roma', 'Maxim', 'Dmitriy']

In [8]:
best_func('NoneName', [])

['NoneName']

In [9]:
best_func('Anton, Liza')

['Roma', 'Maxim', 'Dmitriy', 'Anton', 'Liza']